## Build Audio Vectors
Now that the labels have been extracted, we'll use the compiled csv (df_iemocap.csv) to split the original wav files into multiple frames

In [1]:
# Try for one file first
import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import pickle

import IPython.display
import librosa.display
ms.use('seaborn-muted')
%matplotlib inline

In [2]:
file_path = 'C:/Users/HP/Multimodal/IEMOCAP/IEMOCAP_full_release/Session1/dialog/wav/Ses01F_impro01.wav'

y, sr = librosa.load(file_path, sr=44100)
y, sr

(array([ 0.41168308,  0.509431  ,  0.3655668 , ..., -0.3147118 ,
        -0.16223246, -0.04824252], dtype=float32),
 44100)

## Loop through all the files

In [3]:
import pandas as pd
import math

labels_df = pd.read_csv('C:/Users/HP/Multimodal/pre-processed/df_iemocap.csv')
iemocap_dir = 'C:/Users/HP/Multimodal/IEMOCAP/IEMOCAP_full_release/'

,start_time,end_time,wav_file,emotion,val,act,dom
0,6.2901,8.2357,Ses01F_impro01_F000,neu,2.5,2.5,2.5
1,10.0100,11.3925,Ses01F_impro01_F001,neu,2.5,2.5,2.5
2,14.8872,18.0175,Ses01F_impro01_F002,neu,2.5,2.5,2.5
3,19.2900,20.7875,Ses01F_impro01_F003,xxx,2.5,3.0,3.0
4,21.3257,24.7400,Ses01F_impro01_F004,xxx,2.5,3.0,2.5
...,...,...,...,...,...,...,...
10034,236.5700,244.8300,Ses05M_script03_2_M041,ang,1.0,4.5,5.0
10035,244.8400,246.5800,Ses05M_script03_2_M042,ang,1.0,4.5,4.5
10036,246.5900,248.8300,Ses05M_script03_2_M043,ang,1.5,4.0,4.5
10037,255.8600,260.3300,Ses05M_script03_2_M044,ang,1.0,5.0,5.0


The following cells take some time until completely executed

In [8]:
sr = 44100
audio_vectors = {}
for sess in range(1, 6):  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = '{}Session{}/dialog/wav/'.format(iemocap_dir, sess)
    orig_wav_files = os.listdir(wav_file_path)
    for orig_wav_file in tqdm(orig_wav_files):
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            orig_wav_file, file_format = orig_wav_file.split('.')
            for index, row in labels_df[labels_df['wav_file'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector
        except:
            print('An exception occured for {}'.format(orig_wav_file))
    with open('C:/Users/HP/Multimodal/pre-processed/audio_vectors_{}.pkl'.format(sess), 'wb') as f:
        pickle.dump(audio_vectors, f)

 26%|█████████████████████▍                                                             | 8/31 [00:04<00:13,  1.75it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_3796\2336608295.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


An exception occured for Ses02F_script01_1.pk


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.38it/s]
